In [1]:
#importing the required modules
import os
import time
import numpy as np
import pandas as pd 
from cmapPy.pandasGEXpress.parse import parse

In [63]:
class Create_dataset:
    #class with five method variables
    def __init__(self, directory):
        self.directory = directory
        
    def sig_info_dataframe(self, filename):
        """Create the raw dataset."""
        filepath = os.path.join(self.directory + filename)
        sig_info_df = pd.read_csv(filepath,
                               sep = '\t',
                               header = 0,
                               index_col = 0,
                               low_memory = False)
        return sig_info_df
    
    def filter_shRNA(self, sig_info_df, filter_keyword):
        """Filter the raw dataset based on perturbation type (sh_rna is needed in this case)."""
        filter_to_be_applied = sig_info_df['pert_type'] == filter_keyword
        filtered_df = sig_info_df[filter_to_be_applied].copy()
        return filtered_df
    
    def filter_landmark_genes(self, gene_info_filename):
        """Filter the perturbation type filtered dataset based on the landmark genes."""
        filepath = os.path.join(self.directory + gene_info_filename)
        gene_ids = pd.read_csv(filepath,
                          sep = '\t',
                          header = 0,
                          index_col = 0)
        filtering = gene_ids['pr_is_lm']==1
        gene_ids = gene_ids[filtering]
        gene_ids = gene_ids['pr_gene_symbol']
        return gene_ids
    
    def create_final_dataset(self, filename, sh_rna_filtered, landmark_filtered, gene_id):
        """Create the expression table based on the data that were produced with the previous two steps."""
        global dataframe_lists
        filepath = os.path.join(self.directory + filename)
        filtering = sh_rna_filtered['pert_iname'] == gene_id
        samples = sh_rna_filtered.index[filtering]
        gene_ids = landmark_filtered
        try:
            gene_ids.index = gene_ids.index.astype(str)
            expression = parse(filepath,
                              cid = samples,
                              rid = gene_ids.index).data_df.T[gene_ids.index]
        except:
            gene_ids.index = gene_ids.index.astype(int)
            expression = parse(filepath,
                              cid = samples,
                              rid = gene_ids.index).data_df.T[gene_ids.index]
        dataframe_lists.append(expression)
        
    def writing_output(self, output_directory, output_filename, dataframe_lists):
        """Create the output file."""
        output_filepath = os.path.join(output_directory + output_filename)
        concatanated = pd.concat(dataframe_lists)
        concatanated.to_csv(output_filepath, sep = ',')

In [64]:
#Define the global variable for the class
dataframe_lists = []

In [65]:
#Instantiation of the class. Create the raw dataset.
GSE92742 = Create_dataset(r'D:\balint\perturb_go\data\GSE92742/')
sig_info_file = 'GSE92742_Broad_LINCS_sig_info.txt'
sig_info = GSE92742.sig_info_dataframe(sig_info_file)
sig_info.head()

,pert_id,pert_iname,pert_type,cell_id,pert_dose,pert_dose_unit,pert_idose,pert_time,pert_time_unit,pert_itime,distil_id
sig_id,,,,,,,,,,,
AML001_CD34_24H:A05,DMSO,DMSO,ctl_vehicle,CD34,0.1,%,0.1 %,24,h,24 h,AML001_CD34_24H_X1_F1B10:A05
AML001_CD34_24H:A06,DMSO,DMSO,ctl_vehicle,CD34,0.1,%,0.1 %,24,h,24 h,AML001_CD34_24H_X3_F1B10:A06
AML001_CD34_24H:B05,DMSO,DMSO,ctl_vehicle,CD34,0.1,%,0.1 %,24,h,24 h,AML001_CD34_24H_X1_F1B10:B05|AML001_CD34_24H_X...
AML001_CD34_24H:B06,DMSO,DMSO,ctl_vehicle,CD34,0.1,%,0.1 %,24,h,24 h,AML001_CD34_24H_X3_F1B10:B06
AML001_CD34_24H:BRD-A03772856:0.37037,BRD-A03772856,BRD-A03772856,trt_cp,CD34,0.37037,µM,500 nM,24,h,24 h,AML001_CD34_24H_X1_F1B10:J04|AML001_CD34_24H_X...


In [66]:
#Filtering the raw dataset based on the perturbation type.
sh_rna_filtered = GSE70138.filter_shRNA(sig_info, 'trt_sh.cgs')
sh_rna_filtered.head()

,pert_id,pert_iname,pert_type,cell_id,pert_dose,pert_dose_unit,pert_idose,pert_time,pert_time_unit,pert_itime,distil_id
sig_id,,,,,,,,,,,
CGS001_A375_96H:A2M:1,CGS001-2,A2M,trt_sh.cgs,A375,1.0,µL,1 µL,96,h,96 h,KDC004_A375_96H:TRCN0000006653:-666|KDC004_A37...
CGS001_A375_96H:AARS:1,CGS001-16,AARS,trt_sh.cgs,A375,1.0,µL,1 µL,96,h,96 h,KDB006_A375_96H:TRCN0000045688:-666|KDB006_A37...
CGS001_A375_96H:AATF:1,CGS001-26574,AATF,trt_sh.cgs,A375,1.0,µL,1 µL,96,h,96 h,KDA004_A375_96H:TRCN0000017389:-666|KDA004_A37...
CGS001_A375_96H:ABAT:1,CGS001-18,ABAT,trt_sh.cgs,A375,1.0,µL,1 µL,96,h,96 h,KDA002_A375_96H:TRCN0000034925:-666|KDB006_A37...
CGS001_A375_96H:ABCA1:1,CGS001-19,ABCA1,trt_sh.cgs,A375,1.0,µL,1 µL,96,h,96 h,KDC004_A375_96H:TRCN0000029089:-666|KDC004_A37...


In [67]:
#Filter the perturbation type filtered dataset based on the landmark genes.
landmark_filtered = GSE92742.filter_landmark_genes('GSE92742_Broad_LINCS_gene_info.txt')
landmark_filtered.head()

pr_gene_id
780      DDR1
7849     PAX8
6193     RPS5
23      ABCF1
9552    SPAG7
Name: pr_gene_symbol, dtype: object

In [68]:
gene_ids_list = sh_rna_filtered['pert_iname'].tolist()

In [69]:
#Create the expression table regarding to the pertubation type and landmark genes.
start = time.time()
expression_file = 'GSE92742_Broad_LINCS_Level5_COMPZ.MODZ_n473647x12328.gctx'
for gene_id in gene_ids_list:
    expression = GSE92742.create_final_dataset(expression_file,
                                              sh_rna_filtered,
                                              landmark_filtered,
                                              gene_id)
    if len(dataframe_lists) == 10:
        break
end = time.time()
runtime = (end - start) / 60 #minutes
runtime

0.2672848780949911

In [70]:
#Writing the output.
output_directory = r'D:\balint\perturb_go\data/'
output_filename = 'expression_table.csv'
output = GSE92742.writing_output(output_directory, output_filename, dataframe_lists)